# Part 2 - Mapping Yelp Search Results

## Obective

- For this CodeAlong, we will be working with the Yelp API results from last class. 
- You will load in the .csv.gz of your yelp results and prepare the data for visualization.
- You will use Plotly Express to create an interactive map with all of the results.

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- [Advanced Transformations with Pandas - Part 1](https://login.codingdojo.com/m/376/12529/88086)
- [Advanced Transformations with Pandas - Part 2](https://login.codingdojo.com/m/376/12529/88088)

### Goal

- We want to create a map with every restaurant plotted as a scatter plot with detailed information that appears when we hover over a business
- We will use plotly express's `px.scatter_mapbox` function to accomplish this.
    - https://plotly.com/python/scattermapbox/
    
    - We will need a Mapbox API token for some of the options:
        - https://studio.mapbox.com/
    

# Loading Data from Part 1

In [1]:
## Plotly is not included in your dojo-env
!pip install plotly

In [2]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import json

## importing plotly 
import plotly.express as px

In [5]:
## Load in csv.gz
file = 'Data/final_results_SC_Sushi.csv.gz'

# create a dataframe from the file
df = pd.read_csv(file, compression = 'gzip')

# check the dataframe
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,2jXS4oZkMhAONtd2j7L5Yg,chef-21-sushi-burger-and-korean-bbq-greenville-3,Chef 21 Sushi Burger & Korean BBQ,https://s3-media4.fl.yelpcdn.com/bphoto/DbV4BU...,False,https://www.yelp.com/biz/chef-21-sushi-burger-...,37,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.5,"{'latitude': 34.847671, 'longitude': -82.394229}","['pickup', 'delivery']","{'address1': '500 E McBee Ave', 'address2': 'S...",1.864263e+10,(864) 263-3018,3341.861901,NaN
1,RGRk1ioORwm_FIX8PM732Q,konnichiwa-greenville,Konnichiwa,https://s3-media3.fl.yelpcdn.com/bphoto/p47H0_...,False,https://www.yelp.com/biz/konnichiwa-greenville...,70,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.1,"{'latitude': 34.845952342825115, 'longitude': ...",[],"{'address1': '101 Falls Park Dr', 'address2': ...",1.864252e+10,(864) 252-4436,4184.255183,NaN
2,7cJxOV-ANX1qLThK3yV96w,otto-izakaya-greenville-4,Otto Izakaya,https://s3-media1.fl.yelpcdn.com/bphoto/TdPhFy...,False,https://www.yelp.com/biz/otto-izakaya-greenvil...,448,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.2,"{'latitude': 34.8228218820722, 'longitude': -8...",['delivery'],"{'address1': '15 Market Point Dr', 'address2':...",1.864569e+10,(864) 568-8009,5933.485357,$$
3,zG_XOAFi9Y560WJ1RvghBw,sushi-masa-japanese-restaurant-greenville,Sushi-Masa Japanese Restaurant,https://s3-media1.fl.yelpcdn.com/bphoto/zsRavZ...,False,https://www.yelp.com/biz/sushi-masa-japanese-r...,163,"[{'alias': 'sushi', 'title': 'Sushi Bars'}]",4.4,"{'latitude': 34.8512725830078, 'longitude': -8...",['delivery'],"{'address1': '8590 Pelham Rd', 'address2': 'St...",1.864288e+10,(864) 288-2227,11481.830881,$$
4,Kx1x7Kf6C2gtogQErWSu0A,o-ku-greenville,O-Ku,https://s3-media2.fl.yelpcdn.com/bphoto/7dR0xy...,False,https://www.yelp.com/biz/o-ku-greenville?adjus...,41,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",3.9,"{'latitude': 34.847954222223294, 'longitude': ...",[],"{'address1': '30 W Broad St', 'address2': None...",1.864326e+10,(864) 326-4812,3931.009612,NaN


## Required Preprocessing 

- 1. We need to get the latitude and longitude for each business as separate columns.
- We also want to be able to show the restaurants:
    - name,
    - price range
    - address
    - and if they do delivery or takeout.

### Separating Latitude and Longitude

In [6]:
## use .apply pd.Series to convert a dict to columns
df['coordinates'].apply(pd.Series)

,0
0,"{'latitude': 34.847671, 'longitude': -82.394229}"
1,"{'latitude': 34.845952342825115, 'longitude': ..."
2,"{'latitude': 34.8228218820722, 'longitude': -8..."
3,"{'latitude': 34.8512725830078, 'longitude': -8..."
4,"{'latitude': 34.847954222223294, 'longitude': ..."
...,...
105,"{'latitude': 34.86810664795167, 'longitude': -..."
106,"{'latitude': 34.8499166, 'longitude': -82.3335..."
107,"{'latitude': 34.8279736, 'longitude': -82.3987..."
108,"{'latitude': 34.8715143081717, 'longitude': -8..."


- Why didn't that work???

In [7]:
# separate lat and long from the coordinates col
df[['latitude', 'longitude']] = df['coordinates'].str.split(',', expand = True)

In [8]:
## slice out a single test coordinate
coord = df.loc[0, 'coordinates']
coord

"{'latitude': 34.847671, 'longitude': -82.394229}"

- Its not a dictionary anymore!!! WTF??
    - CSV files cant store iterables (lists, dictionaries) so they get converted to strings.

### Fixing the String-Dictionaries

- The json module has another version of load and dump called `json.loads` and `json.dumps`
    - These are designed to process STRINGS instead of files. 
    
- If we use `json.loads` we can convert our string dictionary into an actual dictionary. 

In [10]:
## Use json.loads on the test coordinate
parsed_coord = json.loads(coord)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

- JSON requires double quotes!

In [11]:
## replace single ' with " 
# df['coordinates'] = df['coordinates'].str.replace("'", '"')
coord = coord.replace("'", '"')
coord

'{"latitude": 34.847671, "longitude": -82.394229}'

In [12]:
## Use json.loads on the test coordinate, again
fixed_coord = json.loads(coord)
fixed_coord

{'latitude': 34.847671, 'longitude': -82.394229}

### Now, how can we apply this same process to the entire column??

In [13]:
## replace ' with " (entire column)
df['coordinates'] = df['coordinates'].str.replace("'", '"')

## apply json.loads
df['coordinates'] = df['coordinates'].apply(json.loads)

df['coordinates'].head()

0     {'latitude': 34.847671, 'longitude': -82.394229}
1    {'latitude': 34.845952342825115, 'longitude': ...
2    {'latitude': 34.8228218820722, 'longitude': -8...
3    {'latitude': 34.8512725830078, 'longitude': -8...
4    {'latitude': 34.847954222223294, 'longitude': ...
Name: coordinates, dtype: object

In [14]:
## slice out a single test coordinate
test_coord = df.loc[0, 'coordinates']
test_coord

{'latitude': 34.847671, 'longitude': -82.394229}

### Using Apply with pd.Series to convert a dictionary column into multiple columns

In [15]:
## use .apply pd.Series to convert a dict to columns => use this code first then check the column names before proceeding with the next code block
df = pd.concat([df, df['coordinates'].apply(pd.Series)], axis = 1)
df.head()

# once you found what's wrong restart and run all then use my code all the way to the end
# lat_long = df['coordinates'].apply(lambda x: pd.Series({'latitude1': x['latitude'], 'longitude1': x['longitude']}))
# lat_long

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price,latitude,longitude,latitude,longitude
0,2jXS4oZkMhAONtd2j7L5Yg,chef-21-sushi-burger-and-korean-bbq-greenville-3,Chef 21 Sushi Burger & Korean BBQ,https://s3-media4.fl.yelpcdn.com/bphoto/DbV4BU...,False,https://www.yelp.com/biz/chef-21-sushi-burger-...,37,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.5,"{'latitude': 34.847671, 'longitude': -82.394229}","['pickup', 'delivery']","{'address1': '500 E McBee Ave', 'address2': 'S...",1.864263e+10,(864) 263-3018,3341.861901,NaN,{'latitude': 34.847671,'longitude': -82.394229},34.847671,-82.394229
1,RGRk1ioORwm_FIX8PM732Q,konnichiwa-greenville,Konnichiwa,https://s3-media3.fl.yelpcdn.com/bphoto/p47H0_...,False,https://www.yelp.com/biz/konnichiwa-greenville...,70,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.1,"{'latitude': 34.845952342825115, 'longitude': ...",[],"{'address1': '101 Falls Park Dr', 'address2': ...",1.864252e+10,(864) 252-4436,4184.255183,NaN,{'latitude': 34.845952342825115,'longitude': -82.4034447},34.845952,-82.403445
2,7cJxOV-ANX1qLThK3yV96w,otto-izakaya-greenville-4,Otto Izakaya,https://s3-media1.fl.yelpcdn.com/bphoto/TdPhFy...,False,https://www.yelp.com/biz/otto-izakaya-greenvil...,448,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.2,"{'latitude': 34.8228218820722, 'longitude': -8...",['delivery'],"{'address1': '15 Market Point Dr', 'address2':...",1.864569e+10,(864) 568-8009,5933.485357,$$,{'latitude': 34.8228218820722,'longitude': -82.3000437297499},34.822822,-82.300044
3,zG_XOAFi9Y560WJ1RvghBw,sushi-masa-japanese-restaurant-greenville,Sushi-Masa Japanese Restaurant,https://s3-media1.fl.yelpcdn.com/bphoto/zsRavZ...,False,https://www.yelp.com/biz/sushi-masa-japanese-r...,163,"[{'alias': 'sushi', 'title': 'Sushi Bars'}]",4.4,"{'latitude': 34.8512725830078, 'longitude': -8...",['delivery'],"{'address1': '8590 Pelham Rd', 'address2': 'St...",1.864288e+10,(864) 288-2227,11481.830881,$$,{'latitude': 34.8512725830078,'longitude': -82.2318115234375},34.851273,-82.231812
4,Kx1x7Kf6C2gtogQErWSu0A,o-ku-greenville,O-Ku,https://s3-media2.fl.yelpcdn.com/bphoto/7dR0xy...,False,https://www.yelp.com/biz/o-ku-greenville?adjus...,41,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",3.9,"{'latitude': 34.847954222223294, 'longitude': ...",[],"{'address1': '30 W Broad St', 'address2': None...",1.864326e+10,(864) 326-4812,3931.009612,NaN,{'latitude': 34.847954222223294,'longitude': -82.40071044533897},34.847954,-82.400710


In [16]:
# once you found what's wrong restart and run all then use my code all the way to the end
lat_long = df['coordinates'].apply(lambda x: pd.Series({'latitude1': x['latitude'], 'longitude1': x['longitude']}))
lat_long

,latitude1,longitude1
0,34.847671,-82.394229
1,34.845952,-82.403445
2,34.822822,-82.300044
3,34.851273,-82.231812
4,34.847954,-82.400710
...,...,...
105,34.868107,-82.347078
106,34.849917,-82.333593
107,34.827974,-82.398701
108,34.871514,-82.255829


In [17]:
## Concatenate the 2 new columns and drop the original.
df = pd.concat((df, lat_long), axis = 1)
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,...,phone,display_phone,distance,price,latitude,longitude,latitude,longitude,latitude1,longitude1
0,2jXS4oZkMhAONtd2j7L5Yg,chef-21-sushi-burger-and-korean-bbq-greenville-3,Chef 21 Sushi Burger & Korean BBQ,https://s3-media4.fl.yelpcdn.com/bphoto/DbV4BU...,False,https://www.yelp.com/biz/chef-21-sushi-burger-...,37,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.5,"{'latitude': 34.847671, 'longitude': -82.394229}",...,1.864263e+10,(864) 263-3018,3341.861901,NaN,{'latitude': 34.847671,'longitude': -82.394229},34.847671,-82.394229,34.847671,-82.394229
1,RGRk1ioORwm_FIX8PM732Q,konnichiwa-greenville,Konnichiwa,https://s3-media3.fl.yelpcdn.com/bphoto/p47H0_...,False,https://www.yelp.com/biz/konnichiwa-greenville...,70,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.1,"{'latitude': 34.845952342825115, 'longitude': ...",...,1.864252e+10,(864) 252-4436,4184.255183,NaN,{'latitude': 34.845952342825115,'longitude': -82.4034447},34.845952,-82.403445,34.845952,-82.403445
2,7cJxOV-ANX1qLThK3yV96w,otto-izakaya-greenville-4,Otto Izakaya,https://s3-media1.fl.yelpcdn.com/bphoto/TdPhFy...,False,https://www.yelp.com/biz/otto-izakaya-greenvil...,448,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.2,"{'latitude': 34.8228218820722, 'longitude': -8...",...,1.864569e+10,(864) 568-8009,5933.485357,$$,{'latitude': 34.8228218820722,'longitude': -82.3000437297499},34.822822,-82.300044,34.822822,-82.300044
3,zG_XOAFi9Y560WJ1RvghBw,sushi-masa-japanese-restaurant-greenville,Sushi-Masa Japanese Restaurant,https://s3-media1.fl.yelpcdn.com/bphoto/zsRavZ...,False,https://www.yelp.com/biz/sushi-masa-japanese-r...,163,"[{'alias': 'sushi', 'title': 'Sushi Bars'}]",4.4,"{'latitude': 34.8512725830078, 'longitude': -8...",...,1.864288e+10,(864) 288-2227,11481.830881,$$,{'latitude': 34.8512725830078,'longitude': -82.2318115234375},34.851273,-82.231812,34.851273,-82.231812
4,Kx1x7Kf6C2gtogQErWSu0A,o-ku-greenville,O-Ku,https://s3-media2.fl.yelpcdn.com/bphoto/7dR0xy...,False,https://www.yelp.com/biz/o-ku-greenville?adjus...,41,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",3.9,"{'latitude': 34.847954222223294, 'longitude': ...",...,1.864326e+10,(864) 326-4812,3931.009612,NaN,{'latitude': 34.847954222223294,'longitude': -82.40071044533897},34.847954,-82.400710,34.847954,-82.400710


In [18]:
# drop original coordinates col
df = df.drop(columns = ['coordinates', 'latitude', 'longitude'])
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,location,phone,display_phone,distance,price,latitude1,longitude1
0,2jXS4oZkMhAONtd2j7L5Yg,chef-21-sushi-burger-and-korean-bbq-greenville-3,Chef 21 Sushi Burger & Korean BBQ,https://s3-media4.fl.yelpcdn.com/bphoto/DbV4BU...,False,https://www.yelp.com/biz/chef-21-sushi-burger-...,37,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.5,"['pickup', 'delivery']","{'address1': '500 E McBee Ave', 'address2': 'S...",1.864263e+10,(864) 263-3018,3341.861901,NaN,34.847671,-82.394229
1,RGRk1ioORwm_FIX8PM732Q,konnichiwa-greenville,Konnichiwa,https://s3-media3.fl.yelpcdn.com/bphoto/p47H0_...,False,https://www.yelp.com/biz/konnichiwa-greenville...,70,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.1,[],"{'address1': '101 Falls Park Dr', 'address2': ...",1.864252e+10,(864) 252-4436,4184.255183,NaN,34.845952,-82.403445
2,7cJxOV-ANX1qLThK3yV96w,otto-izakaya-greenville-4,Otto Izakaya,https://s3-media1.fl.yelpcdn.com/bphoto/TdPhFy...,False,https://www.yelp.com/biz/otto-izakaya-greenvil...,448,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.2,['delivery'],"{'address1': '15 Market Point Dr', 'address2':...",1.864569e+10,(864) 568-8009,5933.485357,$$,34.822822,-82.300044
3,zG_XOAFi9Y560WJ1RvghBw,sushi-masa-japanese-restaurant-greenville,Sushi-Masa Japanese Restaurant,https://s3-media1.fl.yelpcdn.com/bphoto/zsRavZ...,False,https://www.yelp.com/biz/sushi-masa-japanese-r...,163,"[{'alias': 'sushi', 'title': 'Sushi Bars'}]",4.4,['delivery'],"{'address1': '8590 Pelham Rd', 'address2': 'St...",1.864288e+10,(864) 288-2227,11481.830881,$$,34.851273,-82.231812
4,Kx1x7Kf6C2gtogQErWSu0A,o-ku-greenville,O-Ku,https://s3-media2.fl.yelpcdn.com/bphoto/7dR0xy...,False,https://www.yelp.com/biz/o-ku-greenville?adjus...,41,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",3.9,[],"{'address1': '30 W Broad St', 'address2': None...",1.864326e+10,(864) 326-4812,3931.009612,NaN,34.847954,-82.400710


In [19]:
# rename cols 'latitude1', 'longitude1'
df.rename(columns = {'latitude1': 'latitude'}, inplace = True)
df.rename(columns = {'longitude1': 'longitude'}, inplace = True)

# check changes
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,location,phone,display_phone,distance,price,latitude,longitude
0,2jXS4oZkMhAONtd2j7L5Yg,chef-21-sushi-burger-and-korean-bbq-greenville-3,Chef 21 Sushi Burger & Korean BBQ,https://s3-media4.fl.yelpcdn.com/bphoto/DbV4BU...,False,https://www.yelp.com/biz/chef-21-sushi-burger-...,37,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.5,"['pickup', 'delivery']","{'address1': '500 E McBee Ave', 'address2': 'S...",1.864263e+10,(864) 263-3018,3341.861901,NaN,34.847671,-82.394229
1,RGRk1ioORwm_FIX8PM732Q,konnichiwa-greenville,Konnichiwa,https://s3-media3.fl.yelpcdn.com/bphoto/p47H0_...,False,https://www.yelp.com/biz/konnichiwa-greenville...,70,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.1,[],"{'address1': '101 Falls Park Dr', 'address2': ...",1.864252e+10,(864) 252-4436,4184.255183,NaN,34.845952,-82.403445
2,7cJxOV-ANX1qLThK3yV96w,otto-izakaya-greenville-4,Otto Izakaya,https://s3-media1.fl.yelpcdn.com/bphoto/TdPhFy...,False,https://www.yelp.com/biz/otto-izakaya-greenvil...,448,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.2,['delivery'],"{'address1': '15 Market Point Dr', 'address2':...",1.864569e+10,(864) 568-8009,5933.485357,$$,34.822822,-82.300044
3,zG_XOAFi9Y560WJ1RvghBw,sushi-masa-japanese-restaurant-greenville,Sushi-Masa Japanese Restaurant,https://s3-media1.fl.yelpcdn.com/bphoto/zsRavZ...,False,https://www.yelp.com/biz/sushi-masa-japanese-r...,163,"[{'alias': 'sushi', 'title': 'Sushi Bars'}]",4.4,['delivery'],"{'address1': '8590 Pelham Rd', 'address2': 'St...",1.864288e+10,(864) 288-2227,11481.830881,$$,34.851273,-82.231812
4,Kx1x7Kf6C2gtogQErWSu0A,o-ku-greenville,O-Ku,https://s3-media2.fl.yelpcdn.com/bphoto/7dR0xy...,False,https://www.yelp.com/biz/o-ku-greenville?adjus...,41,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",3.9,[],"{'address1': '30 W Broad St', 'address2': None...",1.864326e+10,(864) 326-4812,3931.009612,NaN,34.847954,-82.400710


## Not doing MapBox API exercise because signing up requires my cc

## Creating a Simple Map

### Register for MapBox API

Mapbox API: https://www.mapbox.com/

In [ ]:
## Load in mapbox api credentials from .secret


- Use the plotly express `set_maptbox_acccess_token` function

In [ ]:
## set mapbox token


In [ ]:
## use scatter_mapbox for M.V.P map


### Adding Hover Data

- We want to show the restaurants:
    - name
    - price range
    - address
    - and if they do delivery or takeout.
    
    
- We can use the `hover_name` and `hover_data` arguments for `px.scatter_mapbox` to add this info!

In [ ]:
## add hover_name (name) and hover_data for price,rating,location


### Fixing the Location Column

In [ ]:
## slice out a test address


> Also a string-dictionary...

In [ ]:
## replace ' with "
df['location'] = df['location'].str.replace("'", '"')
df

In [ ]:
## apply json.loads


> Ruh roh....

- Hmm, let's slice out a test_address again and let's write a function to accomplish this instead.
    - We can use try and except in our function to get around the errors.

### Fixing Addresses - with a custom function


In [ ]:
## slice out test address 
test_addr = df.loc[0, 'location']
test_addr

In [ ]:
## write a function to just run json.loads on the address


In [ ]:
## test applying our function


- It worked! Now let's save this as a new column (display_location),
and then let's investigate the businesses that had an "ERROR".

In [ ]:
### save a new display_location column using our function


In [ ]:
## filter for businesses with display_location == "ERROR"


In [ ]:
## slice out a new test address and inspect
test_addr = df.loc[437, 'location']
test_addr

> After some more investigation, we would find a few issues with these "ERROR" rows.
1. They contained None.
2. They contained an apostrophe in the name.
3. ...?

### Possible Fixes (if we care to/have the time)


- Use Regular Expressions to find an fix the display addresses with "'" in them
- Use string split to split on the word display address.
    - Then use string methods to clean up

### Moving Forward without those rows (for now)

In [ ]:
## remove any rows where display_location == 'ERROR'


- We want the "display_address" key from the "display_location" dictionaries.
- We could use a .apply and a lamda to slice out the desired key.

In [ ]:
## use apply and lambda to slice correct key


- Almost done! We want to convert display_address to a string instead a list of strings.
- We can use the string method .join to do so!

In [ ]:
## slice out a test_address


In [ ]:
## test using .join with a "\n"


In [ ]:
## apply the join to every row with a lambda


### Final Map

In [ ]:
## make ourn final map and save as varaible


#### HTML Uses `<br>` instead of `\n`

In [ ]:
## remake the final address column with <br> instead 

## plot the final map

In [ ]:
## use fig.write_html to save map
